In [1]:
import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
# import pandas as ps
import time
import re
import os
import shutil
import glob
import math

# from config import config
import data_helpers
from data_generator import *

In [2]:
TEST_ID = 'U.H.07'
TRAIN_PERCENT= 90
BUFFER_SIZE = 20000
BATCH_SIZE = 48
TEAM_EMBS = 50
LOSS_RAMBDA = 1.0
LEARNING_RATE = 0.00001
MAE_NOT_MCE_P_LOSS = True
TRANSFORMER_DROP = 0.2  #-------------------
TRANSFORMER_LAYERS = 6
TRANSFORMER_HEADS = 6
RESET_HISTORY = False
id_to_ids_filename = 'England-200-1e-07-7300-75-0.8-False'

In [3]:
countryDirPath = "./data/football-data-co-uk/England"
df = data_helpers.get_master_df_from_football_data_co_uk(countryDirPath)

In [ ]:
tokenizer_team = creat_team_tokenizer_uk(df)
print(tokenizer_team.get_vocab_size())

teams = ['Tottenham', 'Arsenal', 'Liverpool', 'what?', 'Tottenham', 'Chelsea', 'e_t', 'Man United', '1234', '[HOME]', '[AWAY]']
teams = [team.strip() for team in [re.sub(r"\s", "_", item) for item in teams]]
teams = " ".join(teams)
encoding = tokenizer_team.encode(teams)
# encoding = tokenizer.encode("")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)

tokenizer_team.decode(encoding.ids)

In [5]:
filepath = os.path.join(os.getcwd(), 'data', 'id_to_ids', id_to_ids_filename + '.json')
id_to_ids = data_helpers.LoadJsonData(filepath)

In [ ]:
lengths = [len(ids) for (tag, label, ids) in id_to_ids.values()]
maxLen = max(lengths)
plt.hist(lengths, np.linspace(0, int(maxLen*1.1), int(maxLen*1.1) + 1))
plt.ylim(plt.ylim())
maxLen = max(lengths)
# plt.plot([maxLen, maxLen], plt.ylim())
plt.title(f'Max length of ids: {maxLen}')

MAX_TOKENS = maxLen

In [ ]:
id_cols = ['id']
Div_cols = ['Div']
Date_cols = ['Date']
Team_cols = ['HomeTeam', 'AwayTeam']
Odds_cols = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']
BB_cols = id_cols + Div_cols + Date_cols + Team_cols + Odds_cols

Half_Goal_cols = ['HTHG', 'HTAG']
Full_Goal_cols = ['FTHG', 'FTAG']
Goal_cols = Half_Goal_cols + Full_Goal_cols
Result_cols = ['HTR', 'FTR']    # A function of Goal_cols, but contribute to better representation.
Shoot_cols = ['HS', 'AS']
ShootT_cols = ['HST', 'AST']
Corner_cols = ['HC', 'AC']
Faul_cols = ['HF', 'AF']
Yellow_cols = ['HY', 'AY']    # H/A Yellow Cards, H/A Red Cards
Red_cols = ['HR', 'AR']    # H/A Yellow Cards, H/A Red Cards
AB_cols = Goal_cols + Result_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols + Red_cols

# underscore_prefixed lists have discontinued columns.
BBAB_cols = BB_cols + AB_cols
_Cols_List_to_Embedd = [Div_cols, Team_cols, Goal_cols, Result_cols]
_Cols_List_to_Standardize = [Odds_cols, Shoot_cols, ShootT_cols, Corner_cols, Faul_cols, Yellow_cols, Red_cols]
_Cols_List_for_Label = [Full_Goal_cols, Odds_cols]
_Label_cols = Full_Goal_cols + Odds_cols

BBAB_cols = BB_cols + AB_cols
base_bbab = list(df.loc[df['id'] == 100000, BBAB_cols].iloc[0, :])
print(base_bbab)

In [ ]:
array = df[Red_cols]
array = np.array(array)
print(array.mean(), array.std())
array = (array - array.mean()) / array.std()
print(array)
print(array.mean(), array.std())

In [ ]:
std_path = os.path.join('./data', 'datasets', id_to_ids_filename + ".json")
std_params = get_standardization_params(df)
print(std_params)
data_helpers.SaveJsonData(std_params, std_path)
std_params = data_helpers.LoadJsonData(std_path)

In [ ]:
ds_path = os.path.join('./data', 'datasets', id_to_ids_filename)

ds = generate_dataset_uk(df, id_to_ids, tokenizer_team, std_params)
tf.data.Dataset.save(ds, ds_path)

ds = tf.data.Dataset.load(ds_path)
len(ds)

In [ ]:
dataset_size = len(ds)
print(dataset_size)
train_size = int(TRAIN_PERCENT/100 * dataset_size)
print(train_size)
test_size = dataset_size - train_size

train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)

print(len(train_ds), len(test_ds), len(ds), len(ds)-len(train_ds)-len(test_ds))

In [12]:
dummy_bbas_tensor = get_dummy_bbas_tensor_uk(df, tokenizer_team, std_params)

In [13]:
def normalize_row(baseId, sequence, base_bb, base_label):
    try:
        seq_len_org = sequence.shape[0]
        nMissings = MAX_TOKENS - seq_len_org    # MAX_TOKENS - tf.shape(sequence)[0].numpy()
        if nMissings > 0:
            block = tf.stack([dummy_bbas_tensor] * nMissings, axis=0)
            sequence = tf.concat([sequence, block], axis=0) 
        # print("sequence 1", sequence.shape)
        # sequence[:, 2] = base[2] - sequence[:, 2]   # get delta days.
        base_bb = base_bb[tf.newaxis, :]    # shape: (seq_len = 1, nFeatures)
        mask = tf.Variable([1] * seq_len_org + [0] * nMissings, dtype=tf.int32) ## DO NOT USE tf.constant !!! unstable.
        mask = mask[:, tf.newaxis] & mask[tf.newaxis, :]
        # print('normalize', sequence.shape, base.shape, mask.shape, mask)
        # seq_len_org = tf.Variable(seq_len_org, dtype=tf.int32)    #--------------------------------- comeback
        return (baseId, sequence, base_bb, base_label, mask, seq_len_org)
    except:
        print('normalize_row exception')
        print('norm 1', sequence.shape, base_bb.shape, base_label.shape, mask.shape, nMissings)
        print('norm 2', baseId, sequence, base_label, mask, nMissings)
        # return (baseId, sequence, base_bb, base_label, mask, seq_len_org)

In [14]:
cnt = 600
for (baseId, sequence, base_bb, base_label) in train_ds:
    z = normalize_row(baseId, sequence, base_bb, base_label)
    cnt -= 1
    if cnt == 0:
        print(baseId)
        print(sequence[0])
        break
# [div 4, teams 8, odds 12, goals 16, results 8, Shoot 2, ShootT 2;    Corner 2, Faul 2, Yellow 2, Red 2]
# 0 Yellows -> -1.25, 0 Reds -> -.2806

In [15]:
class hyperparams:    
    nDivisions = 4 + 1  # E0, E1, E2, E3, and Unknown
    division_embs = 4
    nTeams = tokenizer_team.get_vocab_size()    # including Unknown
    team_embs = TEAM_EMBS
    happen_size = 3 + 1 # 1 for place-holder, for even d_model.
    nResults = 4    # HWin, Draw, AWin, and Unknown
    result_embs = 4
    # odds removed
    d_model = get_std_size()    + division_embs * len(Div_cols) + team_embs * len(Team_cols) \
                                + happen_size - len(Odds_cols)
    batch_size = BATCH_SIZE
    max_tokens = MAX_TOKENS
    num_layers = TRANSFORMER_LAYERS
    num_heads = TRANSFORMER_HEADS
    days_spanning_years = 30
    m365_size = 1
    initial_m365 = 0.9

In [ ]:
print(hyperparams.d_model)

In [17]:
def prepare_batch(baseId, sequence, base_bb, base_label, mask, seq_len_org):
    # target = tf.one_hot(tf.squeeze(tf.cast(base_bbab[:, :, -1], dtype=tf.int32), axis=-1), hyperparams.target_onehot_size)
    
    return (baseId, sequence, base_bb, mask), (base_label, seq_len_org)     # (X, Y)

In [18]:
def normalize_dataset(ds):
    return (
        ds.map(lambda baseId, sequence, base_bb, base_label: tf.py_function(
            func=normalize_row,
            inp=[baseId, sequence, base_bb, base_label],
            Tout=[tf.int32, tf.float32, tf.float32, tf.float32, tf.int32, tf.int32])) #, tf.data.AUTOTUNE == Instability!!!
        )

In [19]:
def make_train_batches(ds):
    return (
        ds
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE)
        .map(prepare_batch, tf.data.AUTOTUNE)
        .cache()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
        )

def make_test_batches(ds):
    return (
        ds
        .batch(BATCH_SIZE)
        .map(prepare_batch, tf.data.AUTOTUNE)
        .cache()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
        )
# Normalize_row
# In: baseId, sequence, base_bb, base_label
# Out: baseId, sequence, base_bb, base_label, mask, seq_len_org

In [20]:
train_ds_path = os.path.join('./data', 'datasets', id_to_ids_filename + '_train_' + str(TRAIN_PERCENT))
if os.path.exists(train_ds_path):
    train_ds = tf.data.Dataset.load(train_ds_path)
else:
    train_ds = normalize_dataset(train_ds)
    tf.data.Dataset.save(train_ds, train_ds_path)

train_batches = make_train_batches(train_ds)

In [21]:
test_ds_path = os.path.join('./data', 'datasets', id_to_ids_filename + '_test_' + str(TRAIN_PERCENT))
if os.path.exists(test_ds_path):
    test_ds = tf.data.Dataset.load(test_ds_path)
else:
    test_ds = normalize_dataset(test_ds)
    tf.data.Dataset.save(test_ds, test_ds_path)

test_batches = make_test_batches(test_ds)

In [28]:
for item in train_batches:
    print(item)
    break

In [23]:
def positional_encoding(memory, depth):
    positions = tf.range(memory.shape[-1], dtype=tf.float32)
    fractional_pos = memory * positions    # fractional position: (batch, fractional position #)
    depth = depth/2
    depths = tf.range(depth, dtype=tf.float32) / depth
    depths = tf.pow(10000.0, depths)    # (depth,)
    angle_rads = fractional_pos[:, :, tf.newaxis] / depths  # (batch, fractional position #, depth)
    # pos_encoding = rearrange([tf.math.sin(angle_rads), tf.math.cos(angle_rads)], 'w b p d -> w h (w t)')
    pos_encoding = tf.concat([tf.math.sin(angle_rads), tf.math.cos(angle_rads)], axis=-1)
    return pos_encoding

In [ ]:
memory = tf.ones((100, 200), dtype=tf.float32) * 0.5
pos_encoding = positional_encoding(memory, depth=512)
# print('pos_encoding', pos_encoding.shape)
pos_encoding = pos_encoding[0, :, :]
# print(pos_encoding.shape)
# Plot the dimensions.
plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

In [25]:
id_cols = ['id']
Div_cols = ['Div']
Date_cols = ['Date']
Team_cols = ['HomeTeam', 'AwayTeam']
Odds_cols = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']
BB_cols = id_cols + Div_cols + Date_cols + Team_cols + Odds_cols

Half_Goal_cols = ['HTHG', 'HTAG']
Full_Goal_cols = ['FTHG', 'FTAG']
Goal_cols = Half_Goal_cols + Full_Goal_cols
Result_cols = ['HTR', 'FTR']    # A function of Goal_cols, but contribute to better representation.
Shoot_cols = ['HS', 'AS']
ShootT_cols = ['HST', 'AST']
Corner_cols = ['HC', 'AC']
Faul_cols = ['HF', 'AF']
Yellow_cols = ['HY', 'AY']    # H/A Yellow Cards, H/A Red Cards
Red_cols = ['HR', 'AR']    # H/A Yellow Cards, H/A Red Cards
AB_cols = Goal_cols + Result_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols + Red_cols

In [26]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, hyperparams, isEncoder=True):
    super().__init__()
    self.isEncoder = isEncoder
    self.division_embedding = tf.keras.layers.Embedding(hyperparams.nDivisions, hyperparams.division_embs, dtype=tf.float32, mask_zero=False) # Learn Unknown
    self.team_embedding = tf.keras.layers.Embedding(hyperparams.nTeams, hyperparams.team_embs, dtype=tf.float32, mask_zero=False) # Learn Unknown
    # self.goal_embedding = tf.keras.layers.Embedding(hyperparams.nGoals, hyperparams.goal_embs, dtype=tf.float32, mask_zero=False) # Learn 0-goal
    # self.result_embedding = tf.keras.layers.Embedding(hyperparams.nResults, hyperparams.result_embs, dtype=tf.float32, mask_zero=False) # Learn Unknown

    self.d_model = hyperparams.d_model
    # print(self.d_model)
    self.position_permuting_dense = tf.keras.layers.Dense(self.d_model)
    self.m365_embedding = tf.keras.layers.Embedding(1, hyperparams.m365_size, mask_zero=False, embeddings_initializer = tf.keras.initializers.Ones())

    self.idx_Days = BB_cols.index('Date')
    assert self.idx_Days == BBAB_cols.index('Date')

  def call(self, x):
    # print('x', x[0].shape, x[1].shape)
    (sequence, base_bb, mask) = x # sob = sequence or base_bb
    sDays = sequence[:, :, self.idx_Days]
    bDays = base_bb[:, :, self.idx_Days]
    # print('days shapes', days.shape)
    
    # BB_cols = id_cols + Div_cols + Date_cols + Team_cols + Odds_cols
    # AB_cols = Goal_cols + Result_cols + Shoot_cols + ShootT_cols + Corner_cols + Faul_cols + Yellow_cols + Red_cols

    sob = None
    if self.isEncoder:
      sob = sequence
    else:
      sob = base_bb

    if self.isEncoder:
      # Extract odds to remove them
      id, div, days, teams, odds, goals, results, remainder \
      = tf.split(sob, [len(id_cols), len(Div_cols), len(Date_cols), len(Team_cols), len(Odds_cols), len(Goal_cols), len(Result_cols),  -1], axis=-1)
      # print('1', remainder[0, 0])
    else:
      # Extract odds to remove them
      id, div, days, teams, odds, remainder \
      = tf.split(sob, [len(id_cols), len(Div_cols), len(Date_cols), len(Team_cols), len(Odds_cols), -1], axis=-1)  
      # print('2', remainder[0, 0])  

    div = self.division_embedding(tf.cast(div, dtype=tf.int32))
    div = tf.reshape(div, [div.shape[0], div.shape[1], -1])
    teams = self.team_embedding(tf.cast(teams, dtype=tf.int32))
    teams = tf.reshape(teams, [teams.shape[0], teams.shape[1], -1])
    if self.isEncoder:
      htGoals, ftGoals = tf.split(goals, [2, 2], axis=-1)
      # print('goals', ftGoals.shape)
      ftGoals = tf.cast(ftGoals, dtype=tf.float32)
      placeHolder = tf.cast(tf.zeros_like(ftGoals[:, :, 1]), dtype=tf.bool)
      happen = (tf.math.greater(ftGoals[..., 0], ftGoals[..., 1]), tf.math.equal(ftGoals[..., 0], ftGoals[..., 1]), tf.math.less(ftGoals[..., 0], ftGoals[..., 1]), placeHolder)
      # happen: (batch, seq, 3)      
      happen = tf.cast(tf.transpose(happen), dtype=tf.float32)  # (batch, nQueries=3)
      happen = tf.einsum('jik', happen)
      # print('happen', happen.shape)
      # goals = self.goal_embedding(tf.cast(goals, dtype=tf.int32))
      # goals = tf.clip_by_value(goals, 0, hyperparams.nGoals)
      # goals = tf.reshape(goals, [goals.shape[0], goals.shape[1], -1])
      # results = self.result_embedding(tf.cast(results, dtype=tf.int32))
      # results = tf.reshape(results, [results.shape[0], results.shape[1], -1])
      # print('results', results.shape)
    
    if self.isEncoder:
      concat = [div, teams, happen, remainder]  # odds removed for this test.
      # print('10', div.shape, teams.shape, odds.shape, goals.shape, results.shape, remainder.shape)
      # print('11', remainder[0,0])
    else:
      concat = [div, teams, remainder]  # odds removed.
      # print('22', remainder[0,0])

    sob = tf.concat(concat, axis=-1)
    # print('sob', sob.shape)
    sob = self.position_permuting_dense(sob)
    # print('33', sob[0][0])

    days_ago = tf.cast(bDays - sDays, dtype=tf.float32) if self.isEncoder else tf.cast(bDays - bDays, dtype=tf.float32)
    
    m365 = self.m365_embedding(tf.zeros_like((hyperparams.m365_size,), dtype=tf.float32)) * hyperparams.initial_m365  # expected shape: (1, hyperparams.remain_365_size)
    m365 = tf.squeeze(m365, axis=0)
    # Let m365 drift freely by commenting this line out, though I am sure it will be within (0, 1)
    # m365 = tf.clip_by_value(m365, 0.01, 1.0)
    memory_alpha = tf.math.pow(m365, 1.0/365) # (hyperparams.m365_size,)
    memory = tf.math.pow(memory_alpha, days_ago[:, :, tf.newaxis])  # decrease as days_ago increase, if memory <= 1.0 as expected.
    memory = tf.reduce_mean(memory, axis=-1)
    # print('pe1', memory.shape)

    pe = positional_encoding(memory, depth=sob.shape[-1]) # (batch, d_model)
    # print('pe2', pe.shape)
    # print('x, pe, d_model', x.shape, pe.shape, self.d_model)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    pe = pe / tf.math.sqrt(tf.cast(sob.shape[-1], tf.float32))
    # print('pe3', pe.shape)
    sob = sob + pe  # comeback

    if self.isEncoder:
      mask = mask
    else:
      mask = mask[:, 0:sob.shape[1], :]

    return (sob, mask)

In [ ]:
pos = PositionalEmbedding(hyperparams, isEncoder=True)

cnt = 2
for z in train_batches:
    (baseId, sequence, base_bb, mask), (base_label, seq_len_org) = z
    cnt -= 1
    if cnt == 0: break
# print('baseId', baseId)
sample_x = (sequence, base_bb, mask)
eSob, eMask = pos.call(sample_x)
print(eSob.shape, eMask.shape)

In [28]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
      super().__init__()
      self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
      self.layernorm = tf.keras.layers.LayerNormalization()   # So the default -1 axix is normalized across. No inter-token operatoin.
      self.add = tf.keras.layers.Add()

class CrossAttention(BaseAttention):
    def call(self, x, context, mask):
      attn_output, attn_scores = self.mha(
          query=x,
          key=context,
          value=context,
          attention_mask=mask,
          return_attention_scores=True)
    
      # Cache the attention scores for plotting later.
      self.last_attn_scores = attn_scores
      x = self.add([x, attn_output])
      x = self.layernorm(x)
      return x
  
class GlobalSelfAttention(BaseAttention): 
    def call(self, x, mask):
      attn_output = self.mha(
          query=x,
          value=x,
          key=x,
          attention_mask=mask)    # intentional inter-token operation
      x = self.add([x, attn_output])  # token-wise
      x = self.layernorm(x)         # normalize across the default -1 axis. No inter-token operatoin.
      return x
  
class CausalSelfAttention(BaseAttention): # mask-agnostic
    def call(self, x):
      attn_output = self.mha(
          query=x,
          value=x,
          key=x,
          use_causal_mask = True)     # look-over mask is generagted and used, in decoder layers
      x = self.add([x, attn_output])  # mask-agnostic
      x = self.layernorm(x)  # normalize across the default -1 axis. No inter-token operatoin.
      return x
  
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
      super().__init__()
      self.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),    # across -1 axis
        tf.keras.layers.Dense(d_model),    # across -1 axis
        tf.keras.layers.Dropout(dropout_rate)    # mask-agnostic
      ])
      self.add = tf.keras.layers.Add()
      self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
      x = self.add([x, self.seq(x)])  # mask-agnostic
      x = self.layer_norm(x)  # normalize across the default -1 axis. No inter-token operatoin.
      return x
  
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
      super().__init__()

      self.self_attention = GlobalSelfAttention(
          num_heads=num_heads,
          key_dim=d_model,
          dropout=dropout_rate)

      self.ffn = FeedForward(d_model, dff)

    def call(self, x, mask):
      # x: (batch, max_tokens, d_model), mask: (batch, max_tokens, max_tokens)
      x = self.self_attention(x, mask)
      x = self.ffn(x)
      return x
  
class Encoder(tf.keras.layers.Layer):
    def __init__(self, hyperparams, dropout_rate=0.1):
      super().__init__()

      self.d_model = hyperparams.d_model
      self.num_layers = hyperparams.num_layers

      self.pos_embedding = PositionalEmbedding(hyperparams)

      self.dropout = tf.keras.layers.Dropout(dropout_rate)
      self.enc_layers = [
          EncoderLayer(d_model=hyperparams.d_model,
                      num_heads=hyperparams.num_heads,
                      dff=hyperparams.d_model * 4,
                      dropout_rate=dropout_rate)
          for _ in range(hyperparams.num_layers)]

    def call(self, x):
      # x = (sequence, base_bb, mask)
      # x[0]: (batch, max_tokens, bbab.len), x[1]: (batch, 1, bb.len), x[2]: (token, max_tokens, max_tokens)
      x, mask = self.pos_embedding(x)  # x: (batch, max_tokens, d_model), mask: (batch, max_tokens, max_tokens)
      x = self.dropout(x)
      for encoder_layer in self.enc_layers:
        x = encoder_layer(x, mask)
      return x  # Shape `(batch_size, seq_len, d_model)`.
  
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
                *,
                d_model,
                num_heads,
                dff,
                dropout_rate=0.1):
      super(DecoderLayer, self).__init__()

      self.causal_self_attention = CausalSelfAttention(
          num_heads=num_heads,
          key_dim=d_model,
          dropout=dropout_rate)
      
      self.cross_attention = CrossAttention(
          num_heads=num_heads,
          key_dim=d_model,
          dropout=dropout_rate)

      self.ffn = FeedForward(d_model, dff)

    def call(self, x, context, cross_attention_mask):
      # x: (batch, 1, d_model), context: (batch, max_tokens, d_mode)
      x = self.causal_self_attention(x=x)
      x = self.cross_attention(x, context, cross_attention_mask)

      # Cache the last attention scores for plotting later
      self.last_attn_scores = self.cross_attention.last_attn_scores

      x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
      return x
  
class Decoder(tf.keras.layers.Layer):
    def __init__(self, hyperparams, dropout_rate=0.1):
      super(Decoder, self).__init__()

      self.d_model = hyperparams.d_model
      self.num_layers = hyperparams.num_layers

      self.pos_embedding = PositionalEmbedding(hyperparams, isEncoder=False)

      self.dropout = tf.keras.layers.Dropout(dropout_rate)
      self.dec_layers = [
          DecoderLayer(d_model=hyperparams.d_model, num_heads=hyperparams.num_heads,
                      dff=hyperparams.d_model * 4, dropout_rate=dropout_rate)
          for _ in range(hyperparams.num_layers)]

      self.last_attn_scores = None

    def call(self, x, context):
      # x = (sequence, base_bb, mask)
      # x[0]: (batch, max_tokens, bbab.len), x[1]: (batch, 1, bb.len), x[2]: (token, max_tokens, max_tokens)
      # context: (batch, max_tokens, d_model)
      # `x` is token-IDs shape (batch, target_seq_len)
      x, ca_mask = self.pos_embedding(x)  # x: (batch, 1, d_model), ca_mask: (batch, 1, max_tokens)     
      x = self.dropout(x)
      for decoder_layer in self.dec_layers:
        x  = decoder_layer(x, context, ca_mask)
      self.last_attn_scores = self.dec_layers[-1].last_attn_scores
      return x
  
class Transformer(tf.keras.Model):
    def __init__(self, hyperparams, dropout_rate=0.1):
      super().__init__()
      self.encoder = Encoder(hyperparams, dropout_rate=dropout_rate)

      self.decoder = Decoder(hyperparams, dropout_rate=dropout_rate)

      self.final_layer = tf.keras.layers.Dense(hyperparams.d_model) #-------------- to modify

    def call(self, inputs):
      # inputs = (sequence, base_bb, mask)
      # sequence: (batch, max_token, aabb), base: (batch, 1, bb), mask: (batch, max_token, max_token)
      x = self.encoder(inputs)  # (batch, max_tokens, d_model)
      x = self.decoder(inputs, x)  # (batch, 1, d_model)
      logits = self.final_layer(x)  # (batch, 1, d_model)
      logits = tf.squeeze(logits, axis=-2)  # (batch, d_model)
      return logits

In [29]:
class Adaptor(tf.keras.layers.Layer):
  def __init__(self, d_output, d_middle, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(d_middle, activation='relu'),    # across -1 axis
      tf.keras.layers.Dense(d_output),    # across -1 axis
      tf.keras.layers.Dropout(dropout_rate)    # mask-agnostic
    ])
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.seq(x)
    x = self.layer_norm(x)  # Do we need this?
    return x

In [30]:
class OneExTwo(tf.keras.Model):
  def __init__(self, dropout_rate=0.1):
    super().__init__()
    self.nQueries = 3
    self.softmax = tf.keras.layers.Softmax(axis=-1)
    self.adaptor = Adaptor(self.nQueries, hyperparams.d_model)
    self.mce = tf.keras.losses.CategoricalCrossentropy(
      from_logits=False, axis=-1, reduction='sum_over_batch_size'
    )

    return
  
  def call(self, input):
    # expected inputs.shape: (batch, hyperparas.d_model)
    happen_logits = self.adaptor(input)     # (batch, nQueries)
    happen_p = self.softmax(happen_logits)
    return happen_p # (batch, nQueries)

  def h_true(self, ftGoals):  # Defines this QGroup. This is for 1X2 QGroup. Derived classes re-define this funciton.
    # ftGoals:  (batch, 2)
    ftGoals = tf.cast(ftGoals, dtype=tf.int32)  # (batch, 2)
    h = (tf.math.greater(ftGoals[..., 0], ftGoals[..., 1]), tf.math.equal(ftGoals[..., 0], ftGoals[..., 1]), tf.math.less(ftGoals[..., 0], ftGoals[..., 1]))
    h = tf.cast(tf.transpose(h), dtype=tf.float32)  # (batch, nQueries)
    return h
  
  def loss(self, ftGoals, happen_p, rambda):
    happen_t = self.h_true(ftGoals)
    loss = self.mce(happen_t, happen_p)   # reduce_sum
    return loss
  
  def profit_back(self, ftGoals, odds, happen_p):
    oh_1_t = tf.math.multiply(odds, self.h_true(ftGoals)) - 1.0
    stake_p = happen_p  # (batch, nQueries)
    # # Choose not to try one-hot, meaning we bet on all queries with different amount.
    stake_p = tf.squeeze(tf.one_hot(tf.nn.top_k(stake_p).indices, tf.shape(stake_p)[-1]), axis=1) # (batch, nQueries)
    profit_back = tf.math.multiply(oh_1_t, stake_p)   # (batch, nQueries)
    profit_back = tf.math.reduce_sum(profit_back, axis=-1)  # (batch,)  sum across queries  
    profit_back = tf.math.reduce_mean(profit_back, axis=None)  # () mean across games.
    return profit_back  # ()
  
  def profit_back_for_best_games(self, ftGoals, odds, happen_p):
    oh_1_t = tf.math.multiply(odds, self.h_true(ftGoals)) - 1.0
    stake_p = happen_p  # (batch, nQueries)
    # # Choose not to try one-hot, meaning we bet on all queries with different amount.
    stake_p = tf.squeeze(tf.one_hot(tf.nn.top_k(stake_p).indices, tf.shape(stake_p)[-1]), axis=1) # (batch, nQueries)
    profit_back = tf.math.multiply(oh_1_t, stake_p)   # (batch, nQueries)
    profit_back = tf.math.reduce_sum(profit_back, axis=-1)  # (batch,)  sum across queries  
    profit_back = tf.math.reduce_mean(profit_back, axis=None)  # () mean across games.
    return profit_back  # ()
    

In [ ]:
a = tf.constant([[3.0, 3.0, 4.0], [0.1, 0.7, 0.3]])
one_hot_a = tf.squeeze(tf.one_hot(tf.nn.top_k(a).indices, tf.shape(a)[-1]), axis=1)
print(one_hot_a)
# one_hot_a = [[ 0.  0.  1.]]

In [32]:
class BettingEPL(tf.keras.Model):
  def __init__(self, hyperparams, loss_rambda=1.0, dropout_rate=0.1):
    super().__init__()
    self.transformer = Transformer(hyperparams, dropout_rate=dropout_rate)
    self.bookies = ['B365', 'Betfair', 'Interwetten', 'William']
    self.qGroup = OneExTwo()
    self.rambda = loss_rambda     #----------------------- Sensitive rambda!!!, Automate optimizing it.

  def call(self, input):
    x = self.transformer(input)
    happen_p = self.qGroup(x)
    return happen_p  # happen_p: (batch, nQueries)
  
  def loss(self, y, happen_p):
    # y: (batch, len(Team_cols)+len(Odds_cols)) 
    # outputs: # [ ( happen_p: (batch, nQueries) ) for _ in self.qGroups ]
    ftGoals, odds = tf.split(y, [2, -1], axis=-1) # (batch, 2), (batch, sum[qGropu.nQueries for qGroup in self.qGroups])
    loss = self.qGroup.loss(ftGoals, happen_p, self.rambda)
    return loss
  
  def back_test(self, y, happen_p):
    # y: (batch, len(Team_cols)+len(Odds_cols)) 
    # outputs: # [ ( shape: (batch, nQueries), shape: (batch, nQueries) ) for _ in self.qGroups ]
    ftGoals, odds = tf.split(y, [2, -1], axis=-1)
    odds_by_qGroup = tf.split(odds, [self.qGroup.nQueries for qQroup in self.bookies], axis=-1)
    scalar_profit_sum_query_mean_batch = [self.qGroup.profit_back(ftGoals, odds, happen_p) 
                   for odds in odds_by_qGroup] # [ () for _ in self.qGroups]
    profit_back = tf.stack(scalar_profit_sum_query_mean_batch, axis=0) # (nQGroups,)
    profit_mean_game_mean_qGroup = tf.math.reduce_mean(profit_back)  # mean across qGroups

    return profit_mean_game_mean_qGroup # profit_mean_game_mean_qGroup

In [33]:
EPL = BettingEPL(hyperparams, loss_rambda = LOSS_RAMBDA, dropout_rate=TRANSFORMER_DROP)

# for z in train_batches:
#     (baseId, sequence, base_bb, mask), (base_label, seq_len_org) = z
#     x = (sequence, base_bb, mask)
#     print(sequence.shape)
#     break
# x = (sequence, base_bb, mask)
# y = EPL(x, training=False)
# print(len(y))
# print(len(y[0]))
# happen_p = y[0]
# print(happen_p.shape)
# # print(profit_p, stake_p)   # profit_p tend to have the same sign in the same batch.

# EPL.summary()

In [34]:
# print(EPL.layers[0].layers[0].get_weights()[6])
# print(EPL.layers[0].layers[1].get_weights()[4])

In [35]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [36]:
# learning_rate = CustomSchedule(hyperparams.d_model)

learning_rate = LEARNING_RATE
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.95, beta_2=0.95, epsilon=1e-9)
# optimizer = tf.keras.optimizers.Adadelta()

In [37]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction='none')

def masked_loss_uk(label, y_pred):
  # lable = (target(batch, 3), base_bb(batch, 1, 9), seq_len(batch, 1)), y_pred: (batch, 3)
  y_true = label[0]   # one_hot: (batch, 3)
  seq_len = label[2]  # (batch, 1)

  mask = y_true != 0 
  loss = loss_object(y_true, y_pred)
  mask = tf.cast(mask, dtype=loss.dtype)
  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask) # eq. sum_loss / batch
  return loss


class recall():
  def __init__(self, name='recall', min_seq_len=5, **kwargs):
    self.min_seq_len = min_seq_len
    self.n = None
    self.recall = None
    self.reset()

  def update_state(self, label, y_pred):
    # lable = (target(batch, 3), base_bb(batch, 1, 9), seq_len(batch,)), y_pred: (batch, 3)
    y_true = label[0]   # one_hot: (batch, 3)
    seq_len = label[2]  # (batch)

    seq_len_mask = tf.cast(seq_len >= self.min_seq_len, dtype=tf.float32)[:, tf.newaxis, tf.newaxis]
    y_true = y_true * seq_len_mask
    y_pred = y_pred * seq_len_mask 

    # print('recall', y_true.shape, y_pred.shape, seq_len_mask.shape)

    true_positives = tf.math.reduce_sum(y_true * y_pred)
    # print('recall', true_positives.numpy())
    possible_positives = tf.math.reduce_sum(y_true)
    recall_keras = true_positives / (possible_positives + 1e-9) #tf.keras.backend.epsilon())
    self.n += 1
    self.recall = self.recall * (self.n-1)/self.n + recall_keras.numpy() / self.n


  def result(self):
    return self.recall
  
  def reset(self):
    self.n = 0
    self.recall = 0.0
  
recall_object = recall(min_seq_len=5)

class precision():
  def __init__(self, name='precision', min_seq_len=5, **kwargs):
    self.min_seq_len = min_seq_len
    self.n = None
    self.precision = None
    self.reset()

  def update_state(self, label, y_pred):
    # lable = (target(batch, 3), base_bb(batch, 1, 9), seq_len(batch,)), y_pred: (batch, 3)
    y_true = label[0]   # one_hot: (batch, 3)
    seq_len = label[2]  # (batch, 1)

    seq_len_mask = tf.cast(seq_len >= self.min_seq_len, dtype=tf.float32)[:, tf.newaxis, tf.newaxis]
    y_true = y_true * seq_len_mask
    y_pred = y_pred * seq_len_mask 

    true_positives = tf.math.reduce_sum(y_true * y_pred)
    predicted_positives = tf.math.reduce_sum(y_pred)
    precision_keras = true_positives / (predicted_positives + 1e-9) #tf.keras.backend.epsilon())
    self.n += 1
    self.precision = self.precision * (self.n-1)/self.n + precision_keras.numpy() / self.n

  def result(self):
    return self.precision
  
  def reset(self):
    self.n = 0
    self.precision = 0.0

precision_object = precision(min_seq_len=5)

In [38]:
@tf.function
def train_step(x, y):
    
    with tf.GradientTape() as tape:
        outputs = EPL(x, training=True)  # [ (batch, 1), (batch, nQueries) for _ in bookies]
        loss_value = EPL.loss(y, outputs)
    
    grads = tape.gradient(loss_value, EPL.trainable_weights)
    optimizer.apply_gradients(zip(grads, EPL.trainable_weights))
    # recall_object.update_state(y, logits)
    # precision_object.update_state(y, logits)
    return loss_value

In [39]:
@tf.function
def test_step(x, y):
    outputs = EPL(x, training=False)  # [ (batch, 1), (batch, nQueries) for _ in bookies]
    loss_value = EPL.loss(y, outputs)
    # recall_object.update_state(y, val_logits)
    # precision_object.update_state(y, val_logits)
    return loss_value

In [40]:
@tf.function
def back_test_step(x, y):
    outputs = EPL(x, training=False)  # [ (batch, 1), (batch, nQueries) for _ in bookies]
    profit_back = EPL.back_test(y, outputs)
    return profit_back

In [41]:
@tf.function
def back_test_step2(x, y):
    outputs = EPL(x, training=False)  # [ (batch, 1), (batch, nQueries) for _ in bookies]
    profit_back = EPL.back_test2(y, outputs)
    return profit_back

In [42]:
# @tf.function  #-------------------- Wierd: no work.
def back_test_with_dataset(datsset):
    n = 0
    profit_back_total = tf.Variable(0.0, dtype=tf.float32)
    for step, ((baseId, sequence, base_bb, mask), (base_label, seq_len_org)) in enumerate(datsset):
        x = (sequence, base_bb, mask); y = base_label
        profit_back = back_test_step(x, y)
        n += 1
        profit_back_total = profit_back_total * (n-1) / n + profit_back / n
    return profit_back_total

In [43]:
# @tf.function  #-------------------- Wierd: no work.
def back_test_with_dataset2(datsset):
    n = 0
    profit_back_total = tf.Variable(0.0, dtype=tf.float32)
    for step, ((baseId, sequence, base_bb, mask), (base_label, seq_len_org)) in enumerate(datsset):
        x = (sequence, base_bb, mask); y = base_label
        profit_back = back_test_step2(x, y)
        n += 1
        profit_back_total = profit_back_total * (n-1) / n + profit_back / n
    return profit_back_total

In [44]:
# @tf.function  #-------------------- Wierd: no work.
def test_with_dataset(datsset):
    n = 0
    val_loss = tf.Variable(0.0, dtype=tf.float32)
    for step, ((baseId, sequence, base_bb, mask), (base_label, seq_len_org)) in enumerate(datsset):
        x = (sequence, base_bb, mask); y = base_label
        n += 1
        val_loss = val_loss * (n-1) / n + test_step(x, y) / n
    return val_loss

In [45]:
class history_class():
    def round_sig(self, x, sig=2):
            return round(x, sig-int(math.floor(math.log10(abs(x))))-1)
    def __init__(self):
        self.history = {'loss': [], 'val_loss': [], 'back100': []}
    def save(self, path):
        data_helpers.SaveJsonData(self.history, path)
    def load(self, path):
        self.history = data_helpers.LoadJsonData(path)
        if self.history is None:
            self.history = {'loss': [], 'val_loss': [], 'back100': []}
    def to_back100(self, back):
        # return float(back * 100 if back >= 0 else back)
        return float(back if back >= 0 else back)
    def append(self, loss, val_loss, back):
        self.history['loss'].append(self.round_sig(float(loss), 4))
        self.history['val_loss'].append(self.round_sig(float(val_loss), 4))
        self.history['back100'].append(self.round_sig(self.to_back100(back), 4))
    def len(self):
        assert len(self.history['loss']) == len(self.history['val_loss'])
        assert len(self.history['loss']) == len(self.history['back100'])
        return len(self.history['loss'])
    def get_latest_item(self):
        if self.len() > 0:
            return (self.history['loss'][-1], self.history['val_loss'][-1], self.history['back100'][-1])
        else:
            return (float('inf'), float('inf'), -1.0)

In [46]:
def show_steps(epoch, step, loss, samples_seen):
    # recall = recall_object.result()
    # precision = precision_object.result()
    # print("epoch: {}, step: {}, loss: {}, recall: {}, precision: {}, samples_seen: {}".
    #       format(epoch, step, float(loss_value), recall, precision, (step + 1) * hyperparams.batch_size))
    print("epoch: {}, step: {}, loss: {}, samples_seen: {}          ".
            format(epoch, step, float(loss), samples_seen), end='\r')
    # recall_object.reset()
    # precision_object.reset()

def show_history(history, baseline=0):
    plt.plot(history.history['loss'][baseline:])
    plt.plot(history.history['val_loss'][baseline:])
    if history.len() > 0:
        base1 = min(history.history['loss'])
        base2 = min(history.history['val_loss'])
        base = min(base1, base2)
    else:
        base = 0.0
    plt.plot([base + b100 for b100 in history.history['back100']][baseline:])
    plt.axhline(y=base, color='r', linestyle='-', linewidth=0.8)
    maxBackTest = max(history.history['back100'][baseline:]) if history.len() > baseline else -1.0
    plt.title(TEST_ID + ": max back100: {}, history len: {}".format(maxBackTest, history.len()))
    plt.grid(True)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss', 'val_profit'], loc='lower left')
    plt.show()

In [47]:
checkpointPath = os.path.join('./data', 'checkpoints', TEST_ID + '_weights')
historyPath = os.path.join('./data', 'checkpoints', TEST_ID + '_history.json')

history = history_class()

if RESET_HISTORY:
    files = glob.glob(checkpointPath + "*")         # "*.*" doesn't work
    result = [os.remove(file) for file in files]
    files = glob.glob(historyPath + "*")            # "*.*" doens't work
    result = [os.remove(file) for file in files]
    EPL.save_weights(checkpointPath)
    history.save(historyPath)

try:
    EPL.load_weights(checkpointPath)
except:
    print('Failed to load model weights.')

history.load(historyPath)
# if history.len() <= 0:
#     print('Creating historic baseline...', end='')
#     loss = test_with_dataset(train_batches)
#     val_loss = test_with_dataset(test_batches)
#     back = back_test_with_dataset(test_batches)
#     history.append(loss, val_loss, back)
#     history.save(historyPath)
#     print('done')

def save_checkpoint(loss, val_loss, back):
    (pre_loss, pre_val_loss, pre_back100) = history.get_latest_item()
    if float(history.to_back100(back)) > pre_back100:
        EPL.save_weights(checkpointPath)
    history.append(loss, val_loss, back)
    history.save(historyPath)

In [ ]:
for key, value in history.history.items():
    print(value)

In [ ]:
show_history(history, baseline=0)

In [ ]:
epochs = 500;  prev_loss = float("inf")
for epoch in range(history.len(), epochs):
    start_time = time.time()

    m = 0; epoch_loss = 0.0
    n = 0; loss = tf.Variable(0.0, dtype=tf.float32); samples_seen = 0

    baseId_1 = None
    train_batches = make_train_batches(train_ds)
    for step, ((baseId, sequence, base_bb, mask), (base_label, seq_len_org)) in enumerate(train_batches):
        if baseId_1 is None: baseId_1 = baseId[0]
        # print('train', sequence.shape)
        x = (sequence, base_bb, mask); y = base_label
        batch_loss = train_step(x, y)
        n += 1; loss = loss * (n-1) / n + batch_loss / n
        
        m += 1; epoch_loss = epoch_loss * (m-1)/m + batch_loss / m

        samples_seen += sequence.shape[0]

        if step % 100 == 0:
            show_steps(epoch, step, loss, samples_seen)
            n = 0; loss = 0.0

    show_steps(epoch, step, loss, samples_seen)
    val_loss = test_with_dataset(test_batches)
    back = back_test_with_dataset(test_batches)
    # back2 = back_test_with_dataset2(test_batches)
    save_checkpoint(epoch_loss, val_loss, back)     #------------------------------------------- comeback

    eM365W = EPL.layers[0].layers[0].get_weights()[6]; eM365W = list(tf.reshape(eM365W, (-1,)).numpy())
    # dM365W =EPL.layers[0].layers[1].get_weights()[4]; dM365W = list(tf.reshape(dM365W, (-1,)).numpy())

    print("epoch: {}, loss: {}, val_loss: {}, back_test: {}, baseId_1: {}, memory365: {:.4f}, time taken: {:.0f}s          "
          .format(epoch, float(epoch_loss), float(val_loss), float(back), baseId_1, eM365W[0] * hyperparams.initial_m365, (time.time() - start_time)))